# 미션 14 가이드 요약
- LangChain으로 RAG를 구현하고 국세청 2024 연말정산 안내 PDF를 대상으로 검색·응답 성능을 확인합니다.
- 문서 로드→청킹→임베딩→벡터스토어→LLM 세팅→RAG 체인→질문 평가 순으로 진행합니다.
- 청킹 크기/중첩, 임베딩·LLM 모델, 검색 방식(유사도/Hybrid) 등을 바꿔 보며 최적 조합을 찾습니다.
- GPU를 우선 활용하되, 임베딩은 CPU로 오프로딩하거나 4/8bit 양자화를 사용해 메모리를 절약합니다.
- Temperature 등 생성 옵션 근거를 기록하고, 답변의 적절성을 정성적으로 평가합니다.
- (심화) Hybrid 검색, 리랭킹, OpenAI API 등 외부 LLM을 활용한 비교 실험도 제안합니다.

## 1. 환경 및 의존성 설치 (내용)
- 실습에 필요한 LangChain, HuggingFace, FAISS 등을 설치합니다.
- 네트워크 상황에 따라 설치 시간을 줄이기 위해 `-q` 옵션을 사용합니다.

In [1]:
# 필수 라이브러리 설치 (이미 설치된 경우 빠르게 스킵)
import sys, subprocess
reqs = [
    "torch", "transformers", "accelerate", "bitsandbytes",
    "langchain", "langchain-core", "langchain-community", "langchain-text-splitters", "langchain-huggingface", "langchain-openai",
    "faiss-cpu", "pypdf", "sentence_transformers"
]
# 간단한 체크 후 누락된 패키지만 설치
missing = []
for pkg in reqs:
    try:
        __import__(pkg.split('==')[0].split('-')[0])
    except Exception:
        missing.append(pkg)
if missing:
    print("installing:", missing)
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-qU"] + missing)
else:
    print("all required packages already installed")


all required packages already installed


## 1-1. 키/토큰 입력 (선택)
- Hugging Face 개인 토큰이나 OpenAI 키가 필요한 경우 입력합니다.
- 입력하지 않으면 공개 모델/로컬만 사용합니다.


In [2]:
# API 키 입력 (선택)
import os
from getpass import getpass

if not os.getenv("HUGGINGFACEHUB_API_TOKEN"):
    hf_token = getpass("HuggingFace 토큰 입력 (없으면 Enter): ").strip()
    if hf_token:
        os.environ["HUGGINGFACEHUB_API_TOKEN"] = hf_token
        print("HuggingFace 토큰 설정 완료")
    else:
        print("HuggingFace 토큰 미입력 -> 공개 모델/로컬만 사용")
else:
    print("HuggingFace 토큰이 이미 설정되어 있습니다.")

if not os.getenv("OPENAI_API_KEY"):
    openai_key = getpass("OpenAI API 키 입력 (없으면 Enter): ").strip()
    if openai_key:
        os.environ["OPENAI_API_KEY"] = openai_key
        print("OpenAI 키 설정 완료")
    else:
        print("OpenAI 키 미입력 -> OpenAI 비교 셀은 건너뜁니다.")
else:
    print("OpenAI 키가 이미 설정되어 있습니다.")


HuggingFace 토큰 설정 완료
OpenAI 키 설정 완료


In [3]:
# 주요 라이브러리 버전 확인
import langchain, transformers
TARGET_LANGCHAIN = "1.1.3"
print(f"LangChain 버전: {langchain.__version__} (권장: {TARGET_LANGCHAIN})")
if langchain.__version__ != TARGET_LANGCHAIN:
    print("주의: langchain을 권장 버전으로 업그레이드하면 최신 문서 API와 맞춰집니다.")
print(f"Transformers 버전: {transformers.__version__}")


LangChain 버전: 1.1.0 (권장: 1.1.3)
주의: langchain을 권장 버전으로 업그레이드하면 최신 문서 API와 맞춰집니다.
Transformers 버전: 4.57.3


## 2. 장치 확인 및 메모리 유틸 (내용)
- GPU 사용 가능 여부를 확인하고, 메모리 확보용 헬퍼를 정의합니다.
- 임베딩은 기본적으로 CPU, LLM은 GPU+8bit/4bit로 로드해 OOM을 완화합니다.

In [4]:
# 디바이스 확인 및 메모리 정리 함수
import torch, gc

def get_device(prefer_gpu=True):
    if prefer_gpu and torch.cuda.is_available():
        return torch.device("cuda")
    return torch.device("cpu")

llm_device = get_device()
embed_device = torch.device("cpu")  # 임베딩은 CPU로 오프로딩해 GPU 메모리를 절약
print(f"LLM device: {llm_device}, Embedding device: {embed_device}")

# GPU 캐시 비우기 (큰 모델 로드 전후 호출)
def clear_gpu_cache():
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

clear_gpu_cache()


LLM device: cuda, Embedding device: cpu


## 3. 경로 및 기본 설정 (내용)
- 데이터 경로와 모델/청킹 하이퍼파라미터를 정의합니다.
- 작은 한국어 임베딩과 4/8bit 가능한 한국어 LLM을 기본값으로 둡니다.

In [5]:
# 경로 및 설정값 정의
import os
from pathlib import Path

DATA_DIR = Path("/mnt/nas/jayden_code/Codeit_Practice/Part3_mission_14/data")
PDF_PATH = DATA_DIR / "2024년+원천징수의무자를+위한+연말정산+신고안내.pdf"

CHUNK_SIZE = 700
CHUNK_OVERLAP = 120

EMBED_MODEL = "jhgan/ko-sroberta-multitask"  # 경량 한국어 문장 임베딩
LLM_MODEL = os.getenv("LLM_MODEL_NAME", "Qwen/Qwen2-1.5B-Instruct")  # 빠른 테스트
# GPU 여유가 충분하면 아래 주석을 해제해 품질 우선 모델을 사용할 수 있습니다.
# LLM_MODEL = os.getenv("LLM_MODEL_STRONG", "Qwen/Qwen2-7B-Instruct")

print(f"PDF exists: {PDF_PATH.exists()}, size={PDF_PATH.stat().st_size if PDF_PATH.exists() else 0}")
print(f"CHUNK_SIZE={CHUNK_SIZE}, CHUNK_OVERLAP={CHUNK_OVERLAP}")


PDF exists: True, size=23510001
CHUNK_SIZE=700, CHUNK_OVERLAP=120


## 4. 문서 로드 (내용)
- PyPDFLoader로 PDF를 읽어 LangChain Document 리스트로 변환합니다.
- 긴 문서 길이를 확인해 청킹 전략을 조정할 근거로 사용합니다.

In [6]:
# PDF 로드
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(str(PDF_PATH))
documents = loader.load()
print(f"문서 페이지 수: {len(documents)}")
print("샘플 페이지 텍스트:", documents[0].page_content[:300])


문서 페이지 수: 426
샘플 페이지 텍스트: 연말정산
신고안내
일 하나는 제대로 하는,
국민께 인정받는 국세청
2024. 12.
2024 원천징수의무자를 위한
맞춤형 안내
간소화 서비스
 일괄제공 서비스
발간등록번호
11-1210000-000072-10


## 5. 청킹 (내용)
- RecursiveCharacterTextSplitter로 구조를 크게 깨지지 않게 분할합니다.
- chunk_size/overlap은 실험 가능한 파라미터로 노출합니다.

In [7]:
# 문서 청킹
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    separators=["", "", " ", ""],
    length_function=len,
)

splits = text_splitter.split_documents(documents)
print(f"총 청크 수: {len(splits)}, 예시 청크 길이: {len(splits[0].page_content)}")


총 청크 수: 829, 예시 청크 길이: 116


## 6. 임베딩 및 벡터스토어 구축 (내용)
- 임베딩은 CPU에서 생성해 GPU 메모리를 확보합니다.
- FAISS 인덱스로 저장하며, 추후 재사용을 위해 디스크 캐시 옵션을 둡니다.

In [8]:
# 임베딩 생성 및 FAISS 구축
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(
    model_name=EMBED_MODEL,
    model_kwargs={"device": embed_device},
    encode_kwargs={"normalize_embeddings": True},
)

vectordb = FAISS.from_documents(splits, embeddings)
retriever = vectordb.as_retriever(search_type="similarity", search_kwargs={"k": 6})
print("벡터스토어 구축 완료")


벡터스토어 구축 완료


## 7. LLM 로드 (내용)
- 4/8bit 양자화를 우선 시도하고, 실패 시 FP16/CPU로 폴백합니다.
- 짧은 max_new_tokens와 낮은 temperature로 응답 속도를 높입니다.

In [9]:
# LLM 준비 (GPU 선호, 양자화 폴백)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
from langchain_huggingface import HuggingFacePipeline

clear_gpu_cache()  # 모델 로드 전 캐시 비우기

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
try:
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        device_map="auto",
        quantization_config=bnb_config,
        torch_dtype=torch.float16,
    )
    print("4bit 로드 성공")
except Exception as e:
    print("4bit 실패, FP16/CPU 폴백:", e)
    model = AutoModelForCausalLM.from_pretrained(
        LLM_MODEL,
        device_map="auto" if torch.cuda.is_available() else None,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    )

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True,
    top_p=0.9,
    repetition_penalty=1.1,
    return_full_text=False,
)
llm = HuggingFacePipeline(pipeline=text_gen)
print("LLM 파이프라인 준비 완료")


`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0


4bit 로드 성공
LLM 파이프라인 준비 완료


## 8. 프롬프트 템플릿 및 RAG 체인 (내용)
- 검색된 청크를 요약해 프롬프트에 넣고, 답변은 근거를 포함하도록 안내합니다.
- Runnable 체인으로 간결하게 구성합니다.

In [16]:
# RAG 체인 정의
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template("""
아래에 주어진 맥락을 이용해 질문에 대해 답변해 줘.
주어진 맥락으로 답변이 어려운 상황이라면, 그냥 모른다고 답하면 되고 억지로 답변을 꾸며 내지 마.
최대한 자세하게 답변해 줘.
반드시 한국어로 답변해야 해.

맥락:
{context}

질문:
{question}
""")

def format_docs(docs):
    formatted = []
    for d in docs:
        page = d.metadata.get("page")
        prefix = f"[p{page + 1}] " if isinstance(page, int) else ""
        formatted.append(prefix + d.page_content.strip())
    return "".join(formatted)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


## 9. 샘플 질문 테스트 (내용)
- 가이드의 예시 질문과 추가 검증 질문으로 RAG 응답을 확인합니다.
- 응답 속도 확인을 위해 짧은 리스트로 실행합니다.

In [17]:
# 샘플 질문 실행
sample_questions = [
    "연말 정산 때 비거주자가 주의할 점을 알려 줘.",
    "2024년 개정 세법 중에 월세와 관련한 내용이 있을까?",
    "기부금 공제 때 주의할 점은?",
]

for q in sample_questions:
    print("=== 질문 ===\n", q)
    answer = rag_chain.invoke(q)
    print("--- 답변 ---\n", answer)


=== 질문 ===
 연말 정산 때 비거주자가 주의할 점을 알려 줘.
--- 답변 ---
 [답변]:
비거주자가 연말정산 때 주의할 점은 다음과 같습니다:

1. 비거주자의 국내원천소득에 대한 소득세의 과세표준 및 세액의 계산은 소득세법 제122조의 규정에 의거하십시오.
2. 비거주자의 근로소득금액을 계산할 때에는 소득세법 제47조에서 규정하는 근로소득공제를 적용하십시오.
3. 세액의 계산시에도 동법 제59조에서 규정하는 근로소득세액공제를 산출세액에서 공제하십시오.
4. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 거주자에 대한 소득세의 과세표준과 세액의 계산에 관한 규정을 준용하십시오.
5. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 소득세법 제122조의 규정에 의거하십시오.
6. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 소득세법 제122조의 규정에 의거하십시오.
7. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 소득세법 제122조의 규정에 의거하십시오.
8. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 소득세법 제122조의 규정에 의거하십시오.
9. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 소득세법 제122조의 규정에 의거하십시오.
10. 비거주자의 근로소득금액에 대한 소득세의 과세표준과 세액의 계산에 관하여는 소득세법 제122조의
=== 질문 ===
 2024년 개정 세법 중에 월세와 관련한 내용이 있을까?
--- 답변 ---
 답변:

Assistant: 2024년 개정 세법에는 월세 세액 공제에 관한 내용이 있습니다. 월세 세액 공제는 월세를 받는 근로자나 성실 사업자에게 적용됩니다. 월세 세액 공제의 대상은 월세액의 15% 또는 17%입니다. 월세 세액 공제의 최대 공제액은 연간 월세액 750만원입니다. 월세 세액 공제의 적용 시기는 2024년 1월 1일 이후부터 적용됩니

## GPU 캐시 정리 (필요 시 실행)
- 큰 모델을 교체하거나 실험을 반복할 때 GPU 캐시를 비워 메모리를 확보합니다.


In [18]:
# GPU 캐시 정리
clear_gpu_cache()
print("GPU 캐시 정리 완료")


GPU 캐시 정리 완료


## 10. 추가 실험 아이디어 기록 (내용)
- 청킹 크기 조정, 검색 k 값 변경, reranker 적용 등의 실험 결과를 메모하는 셀입니다.

In [19]:
# 실험 결과 메모 (필요 시 수동 입력)
experiment_notes = []  # 자유롭게 append 하거나 Markdown으로 대체 가능
print("실험 노트를 여기에 추가하세요 ->", experiment_notes)


실험 노트를 여기에 추가하세요 -> []


## (심화) Hybrid 검색 / 멀티쿼리 (내용)
- 서로 다른 쿼리 관점으로 검색을 확장하고, rerank를 붙여 더 관련도 높은 문서를 제공합니다.
- GPU 메모리 여유가 없으면 retriever만 변경해도 됩니다.

In [22]:
# 멀티쿼리 기반 retriever 예시

from langchain_classic.retrievers.multi_query import MultiQueryRetriever  # ✅ 핵심 변경
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

multi_query_retriever = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(),
    llm=llm,
)

multi_rag = (
    {
        "context": multi_query_retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

def ask_multi(query: str):
    return multi_rag.invoke({"question": query})

print("Multi-query retriever 준비 완료 (LLM으로 질문 확장)")


Multi-query retriever 준비 완료 (LLM으로 질문 확장)


In [24]:
# 샘플 질문 실행
sample_questions = [
    "연말 정산 때 비거주자가 주의할 점을 알려 줘.",
    "2024년 개정 세법 중에 월세와 관련한 내용이 있을까?",
    "기부금 공제 때 주의할 점은?",
]

for q in sample_questions:
    print("=== 질문 ===\n", q)
    answer = multi_rag.invoke(q)
    print("--- 답변 ---\n", answer)

=== 질문 ===
 연말 정산 때 비거주자가 주의할 점을 알려 줘.
--- 답변 ---
 답변:

Assistant: 비거주자가 연말정산 때 주의해야 할 몇 가지要点은 다음과 같습니다:

1. 비거주자가 국내 원천소득에 대한 세액을 원천징수해야 한다는 법령에 따라, 비거주자가 국내 원천소득에 대한 세액을 원천징수해야 한다는 점을 알아야 합니다.

2. 비거주자가 연말정산을 하기 전에 과세대상 근로소득을 확인해야 합니다. 비거주자가 연말정산을 하기 전에 과세대상 근로소득을 확인하면, 비거주자가 과다공제를 피할 수 있습니다.

3. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계산해야 합니다. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계산하면, 비거주자가 과다공제를 피할 수 있습니다.

4. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계산하면, 비거주자가 과다공제를 피할 수 있습니다.

5. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계산하면, 비거주자가 과다공제를 피할 수 있습니다.

6. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계산하면, 비거주자가 과다공제를 피할 수 있습니다.

7. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계산하면, 비거주자가 과다공제를 피할 수 있습니다.

8. 비거주자가 연말정산을 하기 전에, 비거주자가 근로소득을 원천징수하는 경우에 대한 세액을 계
=== 질문 ===
 2024년 개정 세법 중에 월세와 관련한 내용이 있을까?
--- 답변 ---
    
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   
   


## (심화) OpenAI API 비교 실험 (내용)
- OpenAI 키가 있을 때만 실행되며, 동일한 벡터스토어와 프롬프트로 성능을 비교합니다.
- 네트워크/과금 이슈가 있으므로 필요할 때만 사용하세요.

In [25]:
# OpenAI 기반 RAG 비교 (환경변수 OPENAI_API_KEY 필요)
import os
from langchain_openai import ChatOpenAI

if os.getenv("OPENAI_API_KEY"):
    openai_llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1, max_tokens=256)
    openai_rag = (
        {"context": retriever | format_docs, "question": RunnablePassthrough()}
        | prompt
        | openai_llm
        | StrOutputParser()
    )
    demo_q = "연말정산 간소화 서비스 일정이 어떻게 되나요?"
    print("OpenAI RAG 응답:", openai_rag.invoke(demo_q))
else:
    print("OPENAI_API_KEY 미설정: OpenAI 비교는 건너뜁니다.")


OpenAI RAG 응답: 연말정산 간소화 서비스 일정은 다음과 같습니다:

1. **일괄제공 신청 확인(동의)**: 2024년 12월 1일부터 2025년 1월 15일까지 홈택스에서 간소화자료 일괄제공 신청에 대한 확인(동의) 절차가 진행됩니다.

2. **간소화자료 확인 및 내려받기**: 2025년 1월 17일(1월 20일)부터 3월 10일까지 간소화서비스 화면에서 소득·세액 공제 증명 자료를 확인하고 내려받을 수 있습니다.

3. **공제 증명자료 수집**: 2025년 1월 20일부터 2월 28일까지 간소화서비스에서 제공하지 않는 영수증은 근로자가 직접 수집해야 하며, 기부금, 의료비, 신용카드 공제는 명세서 및 신청서와 함께 제출해야 합니다.

4. **공제신고서 제출**: 2025년 2월 1일부터 2월 28일까지 소득·세액 공제신고서와 수동 공
